In [1]:
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, to_json, col, udf, explode, lit, coalesce
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import json

In [2]:
spark = SparkSession.builder.appName('json_test').master('local').getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/30 12:47:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pwd = Path().resolve()
pwd
repo_dir = str(pwd).replace('/src', '')
repo_dir = Path(repo_dir).resolve()
# print(type(repo_dir))
# print(repo_dir)
data_dir = Path(repo_dir) / 'data'
# print(data_dir)
clp_path = str(data_dir / 'clp-places.json')
# print(Path(clp_path).is_file())
cogo_path = str(data_dir / 'cogo-colpnts.json')
# print(Path(cogo_path).is_file())
dats_path = str(data_dir / 'dats-places.json')
# print(Path(dats_path).is_file())
okay_path = str(data_dir / 'okay-places.json')
# print(Path(okay_path).is_file())
spar_path = str(data_dir / 'spar-places.json')
# print(Path(spar_path).is_file())

In [4]:
# Read json files
clp_df = spark.read.options(Header=True).json(clp_path)
cogo_df = spark.read.options(Header=True).json(cogo_path)
dats_df = spark.read.options(Header=True).json(dats_path)
okay_df = spark.read.options(Header=True).json(okay_path)
spar_df = spark.read.options(Header=True).json(spar_path)

In [5]:
clp_df.count()
#clp_df.printSchema()
#okay_df.show(1, 0)

259

In [6]:
cogo_df.count()
#cogo_df.printSchema()
#cogo_df.show(2, 0)

276

In [7]:
dats_df.count()
#dats_df.printSchema()
# okay_df.show(2, 0)

145

In [8]:
okay_df.count()
#okay_df.printSchema()
# okay_df.show(2, 0)

169

In [9]:
spar_df.count()
#spar_df.printSchema()


217

In [10]:
temporaryClosures_schema = StructType([
    StructField('from', StringType()),
    StructField('till', StringType())
])

In [11]:
clp_tmp_exists = clp_df.filter("cast(temporaryClosures as string) != '[]'")
clp_tmp_not_exists = clp_df.filter("cast(temporaryClosures as string) = '[]'")
print(clp_tmp_exists.count())
print(clp_tmp_not_exists.count())

8
251


In [12]:
# exploding temporaryClosures column for clp which has value
# clp_tmp_clos_df = clp_df.withColumn('temporaryClosures_exp', explode(col('temporaryClosures')))
clp_tmp_exists_exp = clp_tmp_exists.withColumn('temporaryClosures', explode(col('temporaryClosures')))
print(clp_tmp_exists_exp.count())

clp_tmp_exists_exp = clp_tmp_exists_exp.withColumn('temporaryClosure_from', col('temporaryClosures.from')) \
.withColumn('temporaryClosure_till', col('temporaryClosures.till')) \
.drop('temporaryClosures')

clp_tmp_not_exists = clp_tmp_not_exists.withColumn('temporaryClosure_from', lit('')) \
.withColumn('temporaryClosure_till', lit('')) \
    .drop('temporaryClosures')
#clp_tmp_exists_exp.printSchema()
#clp_tmp_not_exists.printSchema()

8


In [13]:
# Union exists and not exists clp datasets
clp_union_df = clp_tmp_exists_exp.union(clp_tmp_not_exists)
clp_union_df.count()
#clp_union_df.printSchema()

# exploding handoverservices
#clp_final_df = clp_union_df.withColumn('handoverServices', explode(col('handoverServices')))
clp_final_df = clp_union_df
#clp_final_df.printSchema()
print(clp_final_df.count())

259


In [14]:
cogo_tmp_df = cogo_df.withColumn('temporaryClosure_from', lit('')) \
    .withColumn('temporaryClosure_till', lit(''))
cogo_tmp_df.count()

276

In [21]:
cogo_final_df = cogo_tmp_df.drop('temporaryClosures')
cogo_final_df.count()

276

In [22]:
clp_cogo = clp_final_df.union(cogo_final_df)

In [23]:
clp_cogo.show(2,0)
clp_cogo.count()

+---------------------------------------------------+--------+-------------------+-----------------+----------------------+--------------------------------+--------+---------------------------------------------------+-------+---------------------------------------------------------------------------+-------------------+-------------------------------------------------+-------------------+------------+---------------------+---------------------+
|address                                            |branchId|commercialName     |ensign           |geoCoordinates        |handoverServices                |isActive|moreInfoUrl                                        |placeId|placeSearchOpeningHours                                                    |placeType          |routeUrl                                         |sellingPartners    |sourceStatus|temporaryClosure_from|temporaryClosure_till|
+---------------------------------------------------+--------+-------------------+-----------------+--

535

In [25]:
# okay


okay_tmp_exists = okay_df.filter("cast(temporaryClosures as string) != '[]'")
okay_tmp_not_exists = okay_df.filter("cast(temporaryClosures as string) = '[]'")
print(okay_tmp_exists.count())
print(okay_tmp_not_exists.count())


# exploding temporaryClosures column for okay which has value
# okay_tmp_clos_df = clp_df.withColumn('temporaryClosures_exp', explode(col('temporaryClosures')))
okay_tmp_exists_exp = okay_tmp_exists.withColumn('temporaryClosures', explode(col('temporaryClosures')))
print(okay_tmp_exists_exp.count())
okay_tmp_exists_exp = okay_tmp_exists_exp.withColumn('temporaryClosure_from', col('temporaryClosures.from')) \
.withColumn('temporaryClosure_till', col('temporaryClosures.till')) \
.drop('temporaryClosures')
okay_tmp_exists_exp.count()
#okay_tmp_exists_exp.printSchema()
#okay_tmp_not_exists.printSchema()

# transforming not explode dataframe to temp column to struct from array
okay_tmp_not_exists.count()
okay_tmp_not_exists = okay_tmp_not_exists.drop('temporaryClosures')\
    .withColumn('temporaryClosure_from', lit('')) \
    .withColumn('temporaryClosure_till', lit(''))
okay_tmp_not_exists.count()


# Union exists and not exists clp datasets
okay_union_df = okay_tmp_exists_exp.union(okay_tmp_not_exists)
okay_union_df.count()
#okay_union_df.printSchema()

# exploding handoverservices
#okay_final_df = okay_union_df.withColumn('handoverServices', explode(col('handoverServices')))
okay_final_df = okay_union_df
#okay_final_df.printSchema()
print(okay_final_df.count())

3
166
3
169


In [27]:
clp_cogo_okay = clp_cogo.union(okay_final_df)
clp_cogo_okay.count()

704